<a href="https://colab.research.google.com/github/mahb97/Wakeifier/blob/main/Wakeifyer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!nvidia-smi


Sat Sep 27 20:14:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P0             28W /   70W |    3702MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
!pip install -q --upgrade pip
!pip install -q "transformers>=4.43" "accelerate>=0.32" "peft>=0.11" "trl>=0.9" "bitsandbytes>=0.43.1" sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.7 MB/s eta 0:00:00


To load the model conditionally use:

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

base_model_id = "microsoft/Phi-3-mini-4k-instruct"  # or Mistral-7B if you have more VRAM

has_cuda = torch.cuda.is_available()

quant = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
) if has_cuda else None

tok = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    quantization_config=quant,              # only active on CUDA
    torch_dtype=torch.bfloat16 if has_cuda else torch.float32
)


0) Setup

In [1]:
!pip install -q transformers datasets peft accelerate trl sentencepiece bitsandbytes
import os, textwrap, re, unicodedata, pathlib, random
from pathlib import Path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.7 MB/s eta 0:00:00


Chech GPU

In [2]:
import torch, os, platform
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
!nvidia-smi || echo "No NVIDIA GPU visible"


CUDA available: True
Tesla T4
Sat Sep 27 20:07:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------

1) Upload FW txt

In [3]:
raw = Path("FinnegansWake.txt").read_text(encoding="utf-8")

# Normalize unicode; keep Joyce’s punctuation, just tidy whitespace.
def clean(s):
    s = unicodedata.normalize("NFC", s)
    s = s.replace("\r\n","\n")
    s = re.sub(r"[ \t]+"," ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)  # collapse huge gaps
    return s.strip()

cleaned = clean(raw)

# Split into paragraphs; drop very short fragments.
paras = [p.strip() for p in cleaned.split("\n\n")]
paras = [p for p in paras if len(p.split()) >= 6]

print("Paragraphs:", len(paras))
Path("fw_paragraphs.txt").write_text("\n\n".join(paras), encoding="utf-8")


Paragraphs: 1616


1299781

Hugging Face dataset (packed for causal LM)

In [4]:
from datasets import Dataset

ds = Dataset.from_dict({"text": paras})

# For continued pretraining, we just use a 'text' column.
# Pack sequences to ~1.5k tokens for efficiency.


Small base model + LoRA config

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

BASE = "microsoft/Phi-3-mini-4k-instruct"

tok = AutoTokenizer.from_pretrained(BASE, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# 4-bit quantization (CUDA only)
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    BASE,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_cfg,
)

# ✨ critical bits for QLoRA:
model.config.use_cache = False                           # needed with grad checkpointing
model = prepare_model_for_kbit_training(model)           # enables input grads, fixes norms
model.gradient_checkpointing_enable()

lora_cfg = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()  # sanity check: LoRA params should be trainable


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 3,145,728 || all params: 3,824,225,280 || trainable%: 0.0823


In [16]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="wake_lora_dapt",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    learning_rate=2e-4,
    warmup_ratio=0.05,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=250,
    save_strategy="steps",
    save_steps=500,                       # multiple of eval_steps
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    bf16=True,
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    report_to="none",
)


Tokenize + pack (no labels column needed; next-token loss)

In [17]:
from transformers import DataCollatorForLanguageModeling

max_len = 1536

def tokenize(examples):
    return tok(examples["text"], truncation=False)

tokenized = ds.map(tokenize, batched=True, remove_columns=["text"])

# Pack into fixed-length blocks for causal LM (concatenate then chunk)
def group_texts(examples):
    concat = []
    for ids in examples["input_ids"]:
        concat.extend(ids + [tok.eos_token_id])
    # chunk
    result = {"input_ids": [], "attention_mask": []}
    for i in range(0, len(concat), max_len):
        chunk = concat[i:i+max_len]
        if len(chunk) == max_len:
            result["input_ids"].append(chunk)
            result["attention_mask"].append([1]*max_len)
    return result

lm_ds = tokenized.map(group_texts, batched=True)

# Tiny train/valid split
lm_ds = lm_ds.train_test_split(test_size=0.02, seed=42)
train_ds, val_ds = lm_ds["train"], lm_ds["test"]

collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

Map:   0%|          | 0/1616 [00:00<?, ? examples/s]

Map:   0%|          | 0/1616 [00:00<?, ? examples/s]

Train (continued pretraining with LoRA)

smoke run

In [ ]:
from transformers import TrainingArguments, Trainer
import torch

args = TrainingArguments(
    output_dir="wake_lora_dapt_smoke",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,   # fast feedback
    max_steps=40,                    # stop quickly
    learning_rate=2e-4,
    warmup_ratio=0.05,
    logging_strategy="steps",
    logging_steps=1,                 # ← print every step
    eval_strategy="no",
    save_strategy="no",
    bf16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    report_to="none",
    dataloader_num_workers=0,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds.select(range(min(512, len(train_ds)))),  # small slice
    data_collator=collator,
)
trainer.train()


LoRA gives model a Finnegans-Wake “bias” without needing to retrain the whole network & it runs on a free Colab GPU.

In [22]:
from transformers import TrainingArguments, Trainer
import torch

out_dir = "wake_lora_dapt"

args = TrainingArguments(
    output_dir=out_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,     # effective batch ~32
    num_train_epochs=2,
    learning_rate=2e-4,
    warmup_ratio=0.05,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=250,                      # multiple of 500 below
    save_strategy="steps",
    save_steps=500,                      # 2 × 250
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    bf16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collator,
)

trainer.train()
print(trainer.evaluate())
trainer.save_model(out_dir)
tok.save_pretrained(out_dir)
print("Saved to", out_dir)


Step,Training Loss,Validation Loss


{'eval_loss': 4.8898444175720215, 'eval_runtime': 33.7101, 'eval_samples_per_second': 0.178, 'eval_steps_per_second': 0.089, 'epoch': 2.0}
Saved to wake_lora_dapt


Sanity Check

In [21]:
import torch, math
print("CUDA:", torch.cuda.is_available())
print("Train chunks:", len(train_ds), "| Val chunks:", len(val_ds))

per_device = 2
grad_acc = 16
gpus = torch.cuda.device_count() or 1
eff_batch = per_device * grad_acc * gpus
steps_per_epoch = math.ceil(len(train_ds) / eff_batch)
print("Effective batch:", eff_batch, "| Estimated steps/epoch:", steps_per_epoch)


CUDA: True
Train chunks: 266 | Val chunks: 6
Effective batch: 32 | Estimated steps/epoch: 9


In [23]:
# 1) Final eval (optional)
metrics = trainer.evaluate()
print(metrics)

# 2) Save adapter + tokenizer
trainer.save_model("wake_lora_dapt")
tok.save_pretrained("wake_lora_dapt")


{'eval_loss': 4.8898444175720215, 'eval_runtime': 35.5393, 'eval_samples_per_second': 0.169, 'eval_steps_per_second': 0.084, 'epoch': 2.0}


('wake_lora_dapt/tokenizer_config.json',
 'wake_lora_dapt/special_tokens_map.json',
 'wake_lora_dapt/chat_template.jinja',
 'wake_lora_dapt/tokenizer.model',
 'wake_lora_dapt/added_tokens.json',
 'wake_lora_dapt/tokenizer.json')

Quick test generation (uses the model already in memory)

In [24]:
def wake_generate(prompt, max_new_tokens=160, temperature=1.0, top_p=0.95, rep_pen=1.1):
    sys = "You write in a Finnegans Wake-like style while preserving the factual content of the prompt."
    chat = f"<|system|>\n{sys}\n<|user|>\n{prompt}\n<|assistant|>\n"
    inputs = tok(chat, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out_ids = model.generate(
            **inputs, do_sample=True, temperature=temperature, top_p=top_p,
            repetition_penalty=rep_pen, max_new_tokens=max_new_tokens,
            pad_token_id=tok.eos_token_id
        )
    return tok.decode(out_ids[0], skip_special_tokens=True).split("<|assistant|>")[-1].strip()

print(wake_generate("Please confirm receipt of the attached report by Friday."))


You write in a Finnegans Wake-like style while preserving the factual content of the prompt.
 Please confirm receipt of the attached report by Friday.
 Absolutely, like pears on Pearson's platter, post haste peruse shall be performed; punctiliously as Prometheus propounds prior to peering profoundly at penumbra parables provided for personal parsing—proliferated proofs pressed upon porcelain promised before Peary’s Pendulous period poised precipitating procession purveyed particularized providence precluding perturbation or pause. Perforce pending passage past Paladin Ploughman, present pertinence proclaiming peerlessness perpetuates prudential participation with peculiarity pushed persistently towards pronounced professionals permitted proper perfunctory protocol precedence permissible public presentation plus placid passivity paired perfectly providing palpable product


How to reload later (base + LoRA adapter)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

BASE = "microsoft/Phi-3-mini-4k-instruct"

tok = AutoTokenizer.from_pretrained(BASE, use_fast=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    BASE, device_map="auto", torch_dtype=torch.bfloat16
)
model = PeftModel.from_pretrained(base, "wake_lora_dapt")  # your saved adapter folder
model.eval()


permanently merge adapter into the base for export

In [ ]:
merged = model.merge_and_unload()  # may use more VRAM; skip if you stay with PEFT


Save - Zip & download

In [25]:
!zip -r wake_lora_dapt.zip wake_lora_dapt
from google.colab import files; files.download("wake_lora_dapt.zip")


  adding: wake_lora_dapt/ (stored 0%)
  adding: wake_lora_dapt/chat_template.jinja (deflated 60%)
  adding: wake_lora_dapt/adapter_model.safetensors (deflated 9%)
  adding: wake_lora_dapt/adapter_config.json (deflated 56%)
  adding: wake_lora_dapt/checkpoint-18/ (stored 0%)
  adding: wake_lora_dapt/checkpoint-18/chat_template.jinja (deflated 60%)
  adding: wake_lora_dapt/checkpoint-18/scheduler.pt (deflated 62%)
  adding: wake_lora_dapt/checkpoint-18/trainer_state.json (deflated 56%)
  adding: wake_lora_dapt/checkpoint-18/adapter_model.safetensors (deflated 9%)
  adding: wake_lora_dapt/checkpoint-18/adapter_config.json (deflated 56%)
  adding: wake_lora_dapt/checkpoint-18/special_tokens_map.json (deflated 79%)
  adding: wake_lora_dapt/checkpoint-18/rng_state.pth (deflated 26%)
  adding: wake_lora_dapt/checkpoint-18/tokenizer.model (deflated 55%)
  adding: wake_lora_dapt/checkpoint-18/tokenizer.json (deflated 85%)
  adding: wake_lora_dapt/checkpoint-18/added_tokens.json (deflated 62%)
 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Or copy to drive

In [ ]:
from google.colab import drive; drive.mount("/content/drive")
!cp -r wake_lora_dapt /content/drive/MyDrive/

Real trining pass (epoch-based = fewer hiccups)

In [ ]:
from transformers import TrainingArguments, Trainer
import torch

args = TrainingArguments(
    output_dir="wake_lora_dapt",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,   # effective batch ~= 32
    num_train_epochs=2,               # 2; can bump to 3 later
    learning_rate=2e-4,
    warmup_ratio=0.05,
    logging_strategy="steps",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    bf16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    report_to="none",
    dataloader_num_workers=0
)

trainer = Trainer(
    model=model,                 # QLoRA-prepared model from earlier
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collator
)

trainer.train()
print(trainer.evaluate())
trainer.save_model("wake_lora_dapt")
tok.save_pretrained("wake_lora_dapt")

quick generation helper

In [27]:
import torch

def wake_generate(prompt, max_new_tokens=180, temperature=1.1, top_p=0.95, rep_pen=1.1):
    sys = "You write in a Finnegans Wake-like style while preserving the factual content of the prompt."
    chat = f"<|system|>\n{sys}\n<|user|>\n{prompt}\n<|assistant|>\n"
    inputs = tok(chat, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            do_sample=True,
            temperature=temperature, top_p=top_p,
            repetition_penalty=rep_pen,
            max_new_tokens=max_new_tokens,
            pad_token_id=tok.eos_token_id,
        )
    return tok.decode(out_ids[0], skip_special_tokens=True).split("<|assistant|>")[-1].strip()

print(wake_generate("The man is walking to the shop to buy some milk."))


You write in a Finnegans Wake-like style while preserving the factual content of the prompt.
 The man is walking to the shop to buy some milk.
 Ah, there goes our protagonist on his humble daily sojourn for lactum potable's replenishment; an odyssey quite mundane yet indispensably quixotic! Each step taken by this lonesome individual bespeaks tales untold – from chores and sustenance pursuits drenched deeply with life’s ephemeral complexities—as he embarks toward Dairyland extravaganza. His mission: procure a pint that not only satiates pallid hunger within but weaves together moments shared across countless smiles exchanged over breakfast tables amidst familial merriment or solitary silences during nocturnal introspection alike - all tied subtly unto itself as one delves beyond its apparent simplistic superficiality into existential expl



*  Too tame? ↑ temperature to 1.1.
*  Too wild / off-topic? ↓ temperature to 0.8 and ↑ repetition_penalty to 1.15.
